In [ ]:
import requests



In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

soup.find_all('a')
# [<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
#  <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
#  <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

soup.find(id="link3")

In [3]:
!ls ../raw_data

Artist_studies_master.csv batch 3                   batch 6
batch 1                   batch 3 2                 batch 6 2
batch 1.zip               batch 4                   batch 7
batch 2                   batch 4 2
batch 2 2                 batch 5


In [36]:
studies_csv = pd.read_csv('../raw_data/Artist_studies_master.csv')
studies_df = artist_csv.copy()

In [37]:
studies_df.head()

,Artist (last/only)\n(or other effect),Artist (first),Style represented,Complete,User,sgl img folder,cards folder,Batch #,Check x if non-visual artist,Check x if style or other effect,Notes
0,Adams,Ansel,Yes,x,KyrickYoung,x,x,2.0,-,-,NaN
1,Adams,Arthur,Yes,x,sureailabs,x,x,5.0,-,-,NaN
2,Aenami,Alena,Yes,x,sureailabs,x,x,3.0,-,-,NaN
3,af Klint,Hilma,Yes,x,proximasan,x,x,1.0,-,-,NaN
4,Affandi,NaN,Yes,x,sureailabs,x,x,3.0,-,-,NaN


In [38]:
studies_df.columns

Index(['Artist (last/only)\n(or other effect)', 'Artist (first)',
       'Style represented', 'Complete', 'User', 'sgl img folder',
       'cards folder', 'Batch #', 'Check x if non-visual artist',
       'Check x if style or other effect', 'Notes'],
      dtype='object')

In [39]:
studies_df.columns = ['l_name', 'f_name',
       'style_represented', 'complete', 'user', 'sgl_img_folder',
       'cards_folder', 'batch_id', 'non_visual',
       'style_or_effect', 'notes']

In [33]:
artist_df = studies_df[~studies_df['non_visual'].str.contains('x') & ~studies_df['style_or_effect'].str.contains('x')]

In [40]:
artist_df['key'] = artist_df.apply(lambda row: f"{row['f_name']} {row['l_name']}", axis = 1).str.replace(' ', '_')

In [56]:
artist_df['key'][100]

'Annibale_Carracci'

In [42]:
#!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-py3-none-any.whl size=13462 sha256=a05270f8d261c24a3038ba40d9536e01458be65476147b37e43d9f34dbd7b548
  Stored in directory: /Users/alecsharp/Library/Caches/pip/wheels/ed/88/e3/da3d4d73cb91d659488cfa25913b84bbc26febec99d257bce9
Successfully built wikipedia-api


In [45]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('en')

In [82]:
# https://stackoverflow.com/questions/51122413/regex-for-extracting-all-complex-dates-formats-from-a-string-in-python
import re
import calendar

class DateRegex:
    
    def __init__(self):
        self.hello = 'hi'
        # create regex helper strings
        full_months = [month for month in calendar.month_name if month]
        short_months = [d[:3] for d in full_months]
        months = '|'.join(short_months + full_months)
        day = r'\d+'
        year = r'\d+'
        day_or_year = r'\d+(?:\w+)?'
        separator = r'[.,]?\s+'
        # compile regex
        self.regex_expression = re.compile(rf'(?:{day}{sep})?(?:{months}){separator}{day_or_year}(?:{separator}{year})?')
        
    def get_all_dates(self, input_text):
        
        return self.regex_expression.findall(input_text)

date_regex = DateRegex()

['November 3, 1560',
 'July 15, 1609',
 'December 1595',
 'July 8, 1595',
 'April 1606',
 'October 2003']

In [83]:
import numpy as np

birth_dates_list = []
#artist = 'Hubert_Robert'
artist = 'Annibale_Carracci'
wiki_page = wiki_wiki.page(artist)
if wiki_page.exists():
    
    artist_text = wiki_page.text
    dates = date_regex.get_all_dates(artist_text)
    birth_dates_list.append(dates[0])
else:
    birth_dates_list.append(np.nan)
    

In [84]:
artist_text

'Annibale Carracci (Italian pronunciation: [anˈniːbale karˈrattʃi]; November 3, 1560 – July 15, 1609) was an Italian painter and instructor, active in Bologna and later in Rome. Along with his brother and cousin, Annibale was one of the progenitors, if not founders of a leading strand of the Baroque style, borrowing from styles from both north and south of their native city, and aspiring for a return to classical monumentality, but adding a more vital dynamism. Painters working under Annibale at the gallery of the Palazzo Farnese would be highly influential in Roman painting for decades.\n\nEarly career\nAnnibale Carracci was born in Bologna, and in all likelihood was first apprenticed within his family. In 1582, Annibale, his brother Agostino and his cousin Ludovico Carracci opened a painters\' studio, initially called by some the Academy of the Desiderosi (desirous of fame and learning) and subsequently the Incamminati (progressives; literally "of those opening a new way"). Considere

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  engine="text-davinci-002",
  prompt="A table summarizing the fruits from Goocrux:\n\nThere are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy. There are also loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them.\n\n| Fruit | Color | Flavor |",
  temperature=0,
  max_tokens=100,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)